In [2]:
dir_path = "/diskb/houbowei/ray_results/train_2022-01-05_18-22-49/train_6f168_00012_12_batch_size=32,epochs=10,lr=0.1_2022-01-05_18-22-52/checkpoint_000009/checkpoint"

In [3]:
dir_path = '/diskb/houbowei/ray_results/train_2022-01-05_20-09-55/train_64e33_00107_107_batch_size=8,epochs=100,lr=0.001_2022-01-05_20-16-11/checkpoint_000098/checkpoint'

In [4]:
import torch

In [5]:
state_dict = torch.load(dir_path)

In [6]:
from model import DictNet

In [7]:
model = DictNet()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
model.load_state_dict(state_dict[0])

<All keys matched successfully>

In [9]:
from transformers import AutoTokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [11]:
word = "owl"

In [12]:
import pandas as pd

In [13]:
wordnet = pd.read_csv("./wordnet_bert_common_words.csv")

In [14]:
owl_definition = wordnet[wordnet.words == 'owl']

In [15]:
definition = str(owl_definition.definition.values).strip("''[]")

In [16]:
owl_definition

,norms,words,definition,embeddings_ids,word_ids
9687,1.372753,owl,nocturnal bird of prey with hawk-like beak and...,13547,13547


In [17]:
input = tokenizer(definition, return_tensors='pt')

In [18]:
embeddings = model(**input, word_ids=torch.tensor(13547))['pred_embed']

/diskb/houbowei/clever_nlp/model.py:65: UserWarning: Using a target size (torch.Size([768])) that is different to the input size (torch.Size([1, 768])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(


In [19]:
def cosine_recall(example, original_embeddings, K=10):
    example_embedding = original_embeddings[tokenizer.convert_tokens_to_ids(example)]
    # calculate cosine distance
    cosine_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-08)

    cosine_similarity_of_example = cosine_sim(original_embeddings, example_embedding)

    return tokenizer.convert_ids_to_tokens(cosine_similarity_of_example.argsort(descending=True)[:K])

In [22]:
cosine_recall('butte', model.embedding_weight)

['butte',
 '1628',
 '1641',
 '1790',
 '1738',
 '1740',
 '1760',
 '1635',
 '1718',
 '1711']

In [23]:
def get_pred_embeddings(word, wordnet, tokenizer, model):
    word_definition = wordnet[wordnet.words==word]
    definition = str(word_definition.definition.values).strip("'[]")
    model_inputs = tokenizer(definition, return_tensors='pt')
    word_id = torch.tensor(word_definition.word_ids.values)
    model.eval()
    embeddings = model(**model_inputs, word_ids=word_id)['pred_embed']
    return embeddings

get_pred_embeddings('owl', wordnet, tokenizer, model)

tensor([[-3.6488e-03, -5.4358e-02, -1.4953e-02, -5.5390e-02, -3.2685e-03,
         -1.3658e-02, -3.1868e-02, -1.9052e-02, -1.6168e-02, -7.0835e-02,
         -3.2250e-02, -7.6980e-02, -7.7197e-02, -1.0638e-02, -5.4567e-02,
         -5.4839e-02, -6.2392e-03, -1.8619e-02, -2.2610e-02, -1.2704e-02,
         -2.3183e-03, -5.7066e-02,  2.8438e-02, -1.2125e-02, -2.8190e-02,
         -3.2450e-03,  4.6704e-03, -2.0359e-02, -4.1028e-02, -6.7739e-02,
         -3.8261e-02, -5.9996e-02, -3.1586e-02, -2.3010e-02, -3.1284e-02,
          2.6140e-02, -2.4668e-02, -6.9533e-02, -6.6217e-02, -3.8179e-02,
          8.7201e-03,  2.3213e-03,  1.5194e-02, -3.5676e-02, -7.1127e-03,
          1.7512e-02, -2.4850e-02, -2.1139e-02,  9.9205e-03, -6.1748e-02,
         -4.7415e-02, -9.2251e-03, -2.0584e-02, -5.0991e-02, -7.4479e-02,
         -3.0817e-02,  2.1632e-02, -5.2125e-02, -8.0712e-02, -6.0016e-02,
         -6.3978e-03, -4.3562e-02, -2.6527e-03,  2.0478e-02, -4.0139e-02,
          1.3561e-02, -7.6849e-03,  4.

In [24]:
example_embedding = get_pred_embeddings("butte", wordnet, tokenizer, model)
def norm_recall(example_embeddings, original_embeddings, K=10):
    print(example_embedding.shape)
    distance = ((original_embeddings - example_embedding)**2).sum(1)
    print(distance.shape)
    indexes = distance.argsort()[:K]
    return indexes
indexes = norm_recall(example_embedding, model.embedding_weight)
tokenizer.convert_ids_to_tokens(indexes)

torch.Size([1, 768])
torch.Size([30522])


['mountain',
 'mountains',
 'plateau',
 'valley',
 'hill',
 'cliff',
 'hillside',
 'hills',
 '1983',
 '1982']

In [26]:
def cosine_recall(example_embedding, original_embeddings, K=10):
    # calculate cosine distance
    cosine_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-08)

    cosine_similarity_of_example = cosine_sim(original_embeddings, example_embedding)

    return tokenizer.convert_ids_to_tokens(cosine_similarity_of_example.argsort(descending=True)[:K])
cosine_recall(example_embedding, model.embedding_weight)

['hillside',
 'hilltop',
 'plateau',
 'mountain',
 'mountainous',
 'hilly',
 'cliff',
 '1679',
 '1682',
 'ravine']

In [27]:
model.embedding_weight[13547].norm()

tensor(1.3728)

In [28]:
example_embedding.norm()

tensor(1.1722, grad_fn=<CopyBackwards>)

In [35]:
average_embeddings = 0
example = 'butte'
definition = str(wordnet[wordnet.words==example].definition.values)
print(definition)
for c, word in enumerate(definition.split()):
    average_embeddings += model.embedding_weight[tokenizer.convert_tokens_to_ids(word)]
average_embeddings = average_embeddings / c


['a hill that rises abruptly from the surrounding region; has a flat top and sloping sides']


In [36]:
average_embeddings.norm()

tensor(0.6304)

In [37]:
cosine_recall(average_embeddings, original_embeddings=model.embedding_weight)

['279', '306', '1738', '255', '292', '294', '272', '276', '189', '273']

In [38]:
tokenizer.convert_ids_to_tokens(norm_recall(average_embeddings, original_embeddings=model.embedding_weight))

torch.Size([1, 768])
torch.Size([30522])


['mountain',
 'mountains',
 'plateau',
 'valley',
 'hill',
 'cliff',
 'hillside',
 'hills',
 '1983',
 '1982']

In [39]:
tokenizer.convert_tokens_to_ids('[unused706]')

711

In [40]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("nocturnal bird of prey with hawk-like beak and claws and large head with front-facing eyes"))

[23789,
 4743,
 1997,
 8336,
 2007,
 9881,
 1011,
 2066,
 23525,
 1998,
 10702,
 1998,
 2312,
 2132,
 2007,
 2392,
 1011,
 5307,
 2159]

- [ ] 查找bad cases

In [43]:
wordnet[wordnet.words == 'owl']

,norms,words,definition,embeddings_ids,word_ids
9687,1.372753,owl,nocturnal bird of prey with hawk-like beak and...,13547,13547


In [87]:
definition = str(wordnet[wordnet.words == 'owl'].definition.astype(str).values).strip("['']")
definition

'nocturnal bird of prey with hawk-like beak and claws and large head with front-facing eyes'

In [89]:
# 去掉模长较长的单词。
tokens = tokenizer.tokenize(definition)

In [91]:
ids = tokenizer.convert_tokens_to_ids(tokens)

In [99]:
ids.sort()

In [100]:
ids

[1011,
 1011,
 1997,
 1998,
 1998,
 2007,
 2007,
 2066,
 2132,
 2159,
 2312,
 2392,
 4743,
 5307,
 8336,
 9881,
 10702,
 23525,
 23789]

In [102]:
ids[-3:]

[10702, 23525, 23789]

In [107]:
tokenizer.convert_ids_to_tokens(ids[:5])

['-', '-', 'of', 'and', 'and']

In [124]:
wordnet['words'] = wordnet.words.apply(str)

In [128]:
wordnet.definition = wordnet.definition.apply(str)

In [137]:
wordnet[wordnet.words == 'owl'].definition.values

array(['nocturnal bird of prey with hawk-like beak and claws and large head with front-facing eyes'],
      dtype=object)

In [95]:
model.embedding_weight[ids].norm(dim=1)

tensor([1.4114, 1.1149, 0.8438, 1.3371, 0.8776, 1.2976, 0.9000, 0.9927, 1.4834,
        0.8982, 1.2902, 0.8982, 0.9341, 1.0726, 0.8776, 1.1226, 0.9000, 1.2175,
        1.0178])

In [170]:
tokenizer.convert_ids_to_tokens(norm_recall(embeddings, model.embedding_weight))

torch.Size([1, 768])
torch.Size([30522])


['mountain',
 'mountains',
 'plateau',
 'valley',
 'hill',
 'cliff',
 'hillside',
 'hills',
 '1983',
 '1982']

In [181]:
# 去掉 - 符号
example = "nocturnal bird of prey with hawk like beak and claws and large head with front facing eyes"
def word_definiiton_recall(word, definition):
    print(f'word: {word}, {definiton}')
    inputs = tokenizer(example, return_tensors='pt')

    word_ids = wordnet[wordnet.words == word].word_ids.values
    print(word_ids)
    inputs['word_ids'] = torch.tensor(word_ids)
    model.eval()
    embeddings = model(**inputs)['pred_embed']

    cosine_res = tokenizer.convert_ids_to_tokens(cosine_recall(embeddings, model.embedding_weight))
    norm_res = tokenizer.convert_ids_to_tokens(norm_recall(embeddings, model.embedding_weight))
    print('cosine recall: ', cosine_res)
    print('\n')
    print('norm recall: ', norm_res)
    return cosine_res, norm_res

word_definiiton_recall('owl', example);

word: owl, 'nocturnal bird of prey with hawk-like beak and claws and large head with front-facing eyes'
[13547]
torch.Size([1, 768])
torch.Size([30522])
cosine recall:  ['[unused272]', '[unused316]', '[unused505]', '[unused340]', '[unused412]', '[unused410]', '[unused330]', '[unused399]', '[unused306]', '[unused263]']


norm recall:  ['mountain', 'mountains', 'plateau', 'valley', 'hill', 'cliff', 'hillside', 'hills', '1983', '1982']
